In [1]:
from astropy.io import fits
import numpy as np
from scipy.io.idl import readsav
import subprocess
import string

In [3]:
zips = ['97_v1_savdat', '97_v2_savdata']

for folder in zips:
    
    print('Unpacking a zip file: ' + folder)
    #list unpacked files:
    unpack = subprocess.run(["unzip", folder + ".zip", "-d" +folder, "-x", "Dphi*", "beta*"])
    in_dir = subprocess.run(["ls",folder], text=True, stdout=subprocess.PIPE)
    tmp = in_dir.stdout
    files = tmp.split('.sav\n')[:-1]
    print('Unpacked: ', files)
    print('')
    
    for filename in files:
    
        print('   Reading file ' + filename +'.sav')
        dataset = readsav(folder+'/'+filename +'.sav') #Add verbose=True to get details printed out
        a = dataset['sph_data'].dtype.names 
        #sph_data_names = a
    
        #organise the read sph_data structure into 3D and 2D tables, vectors and integers:
        arrays3 = [a[0],a[1],a[2]]
        arrays2 = [a[16],a[17],a[18]]
        arrays1 = [a[7],a[8],a[9],a[10],a[11],a[12],a[13],a[14],a[15]]
        integers = [a[3],a[4],a[5],a[6]]
        #print('In the sph_data:')
        #print('3D: ', arrays3)
        #print('2D: ', arrays2)
        #print('vector: ', arrays1)
        #print('integer: ', integers)
        
        #Write the integers into primary header:
        hdr = fits.Header()
        hdr['FILENAME'] = filename
        hdr['L0'] = dataset['l0']
        hdr['B0'] = dataset['b0']
        hdr['SOLWIND'] = dataset['solarwind']
        hdr['D'] = dataset['d']
        hdr['BDERIVS'] = dataset['sph_data'].bderivs[0]
        hdr['NR'] = dataset['sph_data'].NR[0]
        hdr['NLAT'] = dataset['sph_data'].NLAT[0]
        hdr['NLON'] = dataset['sph_data'].NLON[0]
        hdr['COMMENT'] = ""
        empty_primary = fits.PrimaryHDU(header=hdr)
        final_HDU = fits.HDUList([empty_primary])
    
        #Write the 3D arrays into images:
        for item in arrays3:
            x = item + " = fits.ImageHDU(dataset['sph_data']."+ item +"[0], name='" + item + "')"
            exec(x)
            y = "final_HDU.append(" + item + ")"
            exec(y)
    
        #Write the 2D arrays into images:
        for item in arrays2:
            x = item + " = fits.ImageHDU(dataset['sph_data']."+ item +"[0], name='" + item + "')"
            exec(x)
            y = "final_HDU.append(" + item + ")"
            exec(y)
        
        #Write the columns into a table:
        for item in arrays1:
            x = item + " = fits.Column(name = '" + item + "', format='D', array = dataset['sph_data']."+ item +"[0])"
            exec(x)
        linekind = fits.Column(name='linekind', array=dataset['linekind'], format='I')
        coldefs = fits.ColDefs([RIX,THETA,PHI,LAT,LON,LONBOUNDS,STR,STTH,STPH,linekind])
        table_HDU = fits.BinTableHDU.from_columns(coldefs)
        final_HDU.append(table_HDU)

        print('   Writing the fits-file.')
        #Write everything to the fits-file:
        final_HDU.writeto(filename +'.fits')
    
        print('   Packing the fits-file.')
        #Pack the created fits-files into a zip:
        pack = subprocess.run(['zip', folder+'_infits.zip', filename+'.fits'])
    
        print('   Deleting the unpacked copy of the fits-file.')
        #Delete the unpacked sav-files
        delete = subprocess.run(['rm', filename+'.fits'])
    
        print('   Deleting the .sav file from the unzipped folder.')
        print('')
        #Delete the unpacked sav-files
        delete = subprocess.run(['rm','-r', folder+'/'+filename+'.sav'])
    
    #Delete the empty folder
    delete = subprocess.run(['rmdir', folder])

Unpacking a zip file: 97_v1_savdat
Unpacked:  ['sph_data_1997-11-04_06:41_before', 'sph_data_1997-11-06_12:37_before']

   Reading file sph_data_1997-11-04_06:41_before.sav
   Writing the fits-file.
   Packing the fits-file.
   Deleting the unpacked copy of the fits-file.
   Deleting the .sav file from the unzipped folder.

   Reading file sph_data_1997-11-06_12:37_before.sav
   Writing the fits-file.
   Packing the fits-file.
   Deleting the unpacked copy of the fits-file.
   Deleting the .sav file from the unzipped folder.

Unpacking a zip file: 97_v2_savdata
Unpacked:  ['sph_data_1997-11-04_05:58_before', 'sph_data_1997-11-06_11:55_before']

   Reading file sph_data_1997-11-04_05:58_before.sav
   Writing the fits-file.
   Packing the fits-file.
   Deleting the unpacked copy of the fits-file.
   Deleting the .sav file from the unzipped folder.

   Reading file sph_data_1997-11-06_11:55_before.sav
   Writing the fits-file.
   Packing the fits-file.
   Deleting the unpacked copy of the